In [1]:
import pandas as pd
import nltk; nltk.download('stopwords')
nltk.download('punkt')

from nltk.tokenize import word_tokenize 

# Regular Expressions
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
# import logging
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

# import warnings
# warnings.filterwarnings("ignore",category=DeprecationWarning)

# from ngram import NGram

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tristandealwis/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
listings = pd.read_csv("listings.csv", usecols=['id', 'name', 'description', 'neighbourhood_cleansed', 'zipcode', 'property_type', 'room_type', 
                                            'price', 'availability_365'])
listings = listings.rename(columns={'id': 'listing_id'})

reviews = pd.read_csv("reviews.csv", usecols=['listing_id', 'comments'])

In [3]:
df = pd.merge(listings, reviews, on='listing_id')

In [4]:
df.describe();

In [30]:
df.head();

In [6]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(df['comments'][1:10000]))

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['the', 'best', 'thing', 'about', 'this', 'place', 'is', 'the', 'location', 'you', 're', 'right', 'in', 'the', 'middle', 'of', 'clinton_hill', 'with', 'easy', 'access', 'to', 'restaurants', 'on', 'franklin', 'avenue', 'and', 'in', 'fort_greene', 'you', 're', 'also', 'close', 'to', 'the', 'and', 'trains', 'so', 'transportation', 'is', 'easy', 'the', 'space', 'is', 'clean', 'and', 'neat', 'if', 'spare', 'it', 'really', 'is', 'much', 'more', 'of', 'one', 'floor', 'apartment', 'than', 'private', 'room', 'as', 'you', 'will', 'only', 'have', 'interactions', 'with', 'the', 'family', 'on', 'the', 'shared', 'stairs', 'and', 'stoop', 'there', 'is', 'however', 'no', 'door', 'at', 'the', 'top', 'of', 'the', 'stairs', 'so', 'sounds', 'travel', 'from', 'the', 'rest', 'of', 'the', 'house', 'lisa_roxanne', 'was', 'kind', 'and', 'accommodating', 'and', 'the', 'children', 'are', 'adorable']


In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
# print('type(data_words_nostops): ', type(data_words_nostops))

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)
# print('type(data_words_bigrams): ', type(data_words_bigrams))
# print(data_words_bigrams)

# Form Trigrams
data_words_trigrams = make_trigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# ! python3 -m spacy download en

nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_trigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['good', 'thing', 'place', 'location', 'right', 'middle', 'easy', 'access', 'restaurant', 'also', 'close', 'train', 'transportation', 'easy', 'space', 'clean', 'neat', 'spare', 'really', 'much', 'floor', 'apartment', 'private', 'room', 'interaction', 'family', 'share', 'stair', 'stoop', 'however', 'door', 'top', 'stair', 'sound', 'kind', 'accommodate', 'child', 'adorable']]


In [11]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 2), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 2), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1)]]


In [25]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha=2,
                                           per_word_topics=True)

In [26]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.054*"nice" + 0.043*"good" + 0.042*"restaurant" + 0.036*"easy" + '
  '0.030*"lot" + 0.025*"bar" + 0.023*"super" + 0.022*"tre" + 0.020*"check" + '
  '0.019*"shop"'),
 (1,
  '0.046*"really" + 0.041*"time" + 0.036*"feel" + 0.031*"make" + 0.026*"home" '
  '+ 0.022*"experience" + 0.022*"thank" + 0.022*"go" + 0.016*"enjoy" + '
  '0.015*"welcome"'),
 (2,
  '0.102*"place" + 0.033*"subway" + 0.031*"perfect" + 0.029*"need" + '
  '0.028*"stay" + 0.027*"well" + 0.024*"close" + 0.024*"walk" + 0.021*"area" + '
  '0.017*"city"'),
 (3,
  '0.116*"great" + 0.088*"stay" + 0.080*"apartment" + 0.058*"host" + '
  '0.054*"location" + 0.041*"would" + 0.040*"clean" + 0.038*"recommend" + '
  '0.036*"comfortable" + 0.024*"neighborhood"'),
 (4,
  '0.029*"room" + 0.023*"apartment" + 0.017*"bed" + 0.016*"night" + '
  '0.014*"get" + 0.013*"bathroom" + 0.011*"small" + 0.011*"little" + '
  '0.010*"arrive" + 0.009*"big"')]


In [27]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.895207837348589

Coherence Score:  0.5726100465166375


In [28]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/tristandealwis/opt/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.050322  0.152097       1        1  24.502445
1      0.126686  0.207517       2        1  21.921427
2     -0.055458  0.053173       3        1  20.864872
3      0.226039 -0.304160       4        1  19.066116
0     -0.347588 -0.108628       5        1  13.645138, topic_info=    Category         Freq       Term        Total  loglift  logprob
46   Default  5189.000000      great  5189.000000  30.0000  30.0000
20   Default  5043.000000      place  5043.000000  29.0000  29.0000
4    Default  4925.000000  apartment  4925.000000  28.0000  28.0000
52   Default  5927.000000       stay  5927.000000  27.0000  27.0000
57   Default  2600.000000       host  2600.000000  26.0000  26.0000
..       ...          ...        ...          ...      ...      ...
12    Topic5  1386.636353       good  2045.980469   1.6028  -3.1431
226   Topic5   271.820190       spot   307.380768   1.8688  -4.7726
214   Topic5   640.738953      check   981.845459   1.5650  -3.9151
141   Topic5   265.920135       many   464.178986   1.4347  -4.7945
7     Topic5   304.969635      close  1497.003784   0.4008  -4.6575

[260 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
243       5  0.994393     accessible
1         4  0.997566    accommodate
290       3  0.991386  accommodation
403       2  0.995781         advice
264       2  0.997099         airbnb
...     ...       ...            ...
193       4  0.597801      wonderful
369       1  0.998456           work
61        1  0.082350          would
61        3  0.005882          would
61        4  0.911245          would

[335 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])